# 项目：评估和清理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。

本实战项目的目的在于练习评估数据干净和整洁度，并且基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。

数据每列的含义如下：
- `InvoiceNo`: 发票号码。6位数，作为交易的唯一标识符。如果这个代码以字母“c”开头，表示这笔交易被取消。
- `StockCode`: 产品代码。5位数，作为产品的唯一标识符。
- `Description`: 产品名称。
- `Quantity`: 产品在交易中的数量。
- `InvoiceDate`: 发票日期和时间。交易发生的日期和时间。
- `UnitPrice`: 单价。价格单位为英镑（£）。
- `CustomerID`: 客户编号。5位数，作为客户的唯一标识符。
- `Country`: 国家名称。客户所居住的国家的名称。

## 读取数据

导入数据分析所需要的库，并通过Pandas的`read_csv`函数，将原始数据文件"e_commerce.csv"里的数据内容，解析为DataFrame，并赋值给变量`original_data`。

In [1]:
import pandas as pd

In [2]:
original_data = pd.read_csv("e_commerce.csv")

In [3]:
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 评估数据

在这一部分，我将对在上一部分建立的`original_data`这个DataFrame所包含的数据进行评估。

评估主要从两个方面进行：结构和内容，即整齐度和干净度。数据的结构性问题指不符合“每列是一个变量，每行是一个观察值，每个单元格是一个值”这三个标准，数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

### 评估数据整齐度

In [4]:
original_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
10477,537237,22271,FELTCRAFT DOLL ROSIE,1,12/6/2010 9:58,5.91,NaN,United Kingdom
411839,572234,23534,STOP FOR TEA WALL ART,3,10/21/2011 14:41,5.95,16293.0,United Kingdom
402532,571502,21922,UNION STRIPE WITH FRINGE HAMMOCK,4,10/17/2011 15:18,7.95,14422.0,United Kingdom
165910,550835,22228,BUNNY WOODEN PAINTED WITH BIRD,4,4/21/2011 10:52,0.85,15034.0,United Kingdom
393321,570818,22720,SET OF 3 CAKE TINS PANTRY DESIGN,2,10/12/2011 12:47,4.95,17841.0,United Kingdom
370282,569134,85159B,"WHITE TEA,COFFEE,SUGAR JARS",12,9/30/2011 14:07,1.95,15411.0,United Kingdom
367792,568908,22077,6 RIBBONS RUSTIC CHARM,12,9/29/2011 13:38,1.95,16653.0,United Kingdom
129649,547386,84988,SET OF 72 PINK HEART PAPER DOILIES,2,3/22/2011 15:52,2.92,NaN,United Kingdom
277955,561201,23080,RED METAL BOX TOP SECRET,2,7/25/2011 15:08,8.25,14692.0,United Kingdom
198521,554054,22082,RIBBON REEL STRIPES DESIGN,1,5/20/2011 15:29,3.29,NaN,United Kingdom


从抽样的10行数据数据来看，数据符合“每列是一个变量，每行是一个观察值，每个单元格是一个值”，具体来看每行是关于某商品的一次交易，每列是交易相关的各个变量，因此不存在结构性问题。

### 评估数据干净度

In [5]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


从输出结果来看，数据共有541909条观察值，而`Description`、`CustomerID`变量存在缺失值。

此外，`InvoiceDate`的数据类型应为日期，`CustomerID`的数据类型应为字符串，应当进行数据格式转换。

#### 评估缺失数据

在了解`Description`存在缺失值后，根据条件提取出缺失观察值。

In [6]:
original_data[original_data["Description"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
1988,536550,85044,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
2024,536552,20950,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
2025,536553,37461,NaN,3,12/1/2010 14:35,0.0,NaN,United Kingdom
2026,536554,84670,NaN,23,12/1/2010 14:35,0.0,NaN,United Kingdom
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom


有1454条交易数据缺失`Description`变量值。

从输出结果来看，这些缺失`Description`的交易数据，`UnitPrice`都为0。为了验证猜想，我们增加筛选条件，看是否存在`Description`变量缺失且`UnitPrice`不为0的数据。

In [7]:
original_data[(original_data["Description"].isnull()) & (original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


筛选出来结果数量为0条，说明缺失`Description`值的数据，同时也不具备有效的`UnitPrice`值。

`Description`表示产品名称，`UnitPrice`表示产品单价，都是进行后续商品交易分析的重要变量。如果它们同时缺失/无效，我们认为数据无法提供有效含义，因此这些后续可以被删除。

`CustomerID`变量同样存在缺失值，因此也根据条件提取出缺失观察值。

In [8]:
original_data[original_data["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,12/1/2010 14:32,1.66,NaN,United Kingdom
1448,536544,21791,VINTAGE HEADS AND TAILS CARD GAME,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1449,536544,21801,CHRISTMAS TREE DECORATION WITH BELL,10,12/1/2010 14:32,0.43,NaN,United Kingdom
1450,536544,21802,CHRISTMAS TREE HEART DECORATION,9,12/1/2010 14:32,0.43,NaN,United Kingdom
1451,536544,21803,CHRISTMAS TREE STAR DECORATION,11,12/1/2010 14:32,0.43,NaN,United Kingdom


`CustomerID`表示客户编号，不是分析畅销商品的必要变量。并且从输出结果来看，有些`CustomerID`缺失的交易数据仍然有效，因此保留此变量为空的观察值。

#### 评估重复数据

根据数据变量的含义来看，虽然`InvoiceNo`、`StockCode`和`CustomerID`都是唯一标识符，但一次交易可能包含多件商品，因此`InvoiceNo`可以存在重复，不同交易可以包含同件商品，因此`StockCode`可以存在重复，顾客可以进行多次交易或下单多个商品，因此`CustomerID`也可以存在重复。

那么针对此数据集，我们无需评估重复数据。

#### 评估不一致数据

不一致数据可能存在于`Country`变量中，我们要查看是否存在多个不同值指代同一国家的情况。

In [9]:
original_data["Country"].value_counts()

United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68
European

从`Country`变量值来看，`"USA"`、`"United States"`均在表示美国，`"United Kingdom"`、`"UK"`、`"U.K."`均在表示英国，因此应该在清洗步骤对这些值进行统一，只保留一个指代值。

#### 评估无效或错误数据

可以通过DataFrame的`describe`方法，对数值统计信息进行快速了解。

In [10]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


从输出结果来看，`Quantity`和`UnitPrice`存在负数，会对后续数值分析产生影响。

因此，我们先筛选出`Quantity`数值为负数的观察值，进一步评估其含义。

In [11]:
original_data[original_data["Quantity"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,12/1/2010 10:24,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,12/1/2010 10:24,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,12/1/2010 12:38,4.25,17897.0,United Kingdom


从筛选结果来看，`Quantity`变量为负数的观察值，`InvoiceNo`似乎均以C开头，表示订单被取消。

但此猜测不一定准确，为了验证猜想，我们增加筛选条件，看是否存在`Quantity`变量为负数且`InvoiceNo`不以C开头的观察值。

In [12]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != "C")]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom
7192,537000,21414,NaN,-22,12/3/2010 15:32,0.0,NaN,United Kingdom
7193,537001,21653,NaN,-6,12/3/2010 15:33,0.0,NaN,United Kingdom
7195,537003,85126,NaN,-2,12/3/2010 15:33,0.0,NaN,United Kingdom
7196,537004,21814,NaN,-30,12/3/2010 15:34,0.0,NaN,United Kingdom
7197,537005,21692,NaN,-70,12/3/2010 15:35,0.0,NaN,United Kingdom


以上猜想被证实错误，因为还存在`Quantity`变量为负数且`InvoiceNo`不以C开头的观察值。

但根据以上输出结果，这些筛选出的观察值的`UnitPrice`观察值均为0，因此增加`UnitPrice`的条件进行验证。

In [13]:
original_data[(original_data["Quantity"] < 0)&(original_data["InvoiceNo"].str[0] != "C")&(original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


根据输出结果，猜想得到验证，当`Quantity`变量为负数的时候，观察值满足以下条件之一：
- `InvoiceNo`以C开头，表示订单被取消
- `UnitPrice`为0，说明单价为0英镑

这些交易数据均不是有效成交数据，不贡献销售，不在后续分析范围内，因此我们将在数据清理步骤，将`Quantity`为负数的观察值删除。

接下来筛选出`UnitPrice`为负数的观察值。

In [14]:
original_data[original_data["UnitPrice"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


从输出结果来看，`UnitPrice`为负数的观察值都是坏账调账，不属于实际商品交易数据，因此也在数据清理步骤中也将其删除。

## 清理数据

根据前面评估部分得到的结论，我们需要进行的数据清理包括：
- 把`InvoiceDate`变量的数据类型转换为为日期时间
- 把`CustomerID`变量的数据类型转换为字符串
- 把`Description`变量缺失的观察值删除
- 把`Country`变量值`"USA"`替换为`"United States"`
- 把`Country`变量值`"UK"`、`"U.K."`替换为`"United Kingdom"`
- 把`Quantity`变量值为负数的观察值删除
- 把`UnitPrice`变量值为负数的观察值删除

为了区分开经过清理的数据和原始的数据，我们创建新的变量`cleaned_data`，让它为`original_data`复制出的副本。我们之后的清理步骤都将被运用在`cleaned_data`上。

In [15]:
cleaned_data = original_data.copy()
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


把`InvoiceDate`变量的数据类型转换为为日期时间类型：

In [ ]:
cleaned_data["InvoiceDate"] = pd.to_datetime(cleaned_data["InvoiceDate"])
cleaned_data["InvoiceDate"]

把`CustomerID`变量的数据类型转换为字符串：

In [ ]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"]

把`CustomerID`变量值结尾的`".0"`删除：

In [ ]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].str.slice(0, -2)
cleaned_data["CustomerID"]

把`Description`变量缺失的观察值删除，并查看删除后该列空缺值个数和：

In [ ]:
cleaned_data.dropna(subset=["Description"], inplace=True)

In [ ]:
cleaned_data["Description"].isnull().sum()

把`Country`变量值`"USA"`替换为`"United States"`，并检查替换后`"USA"`变量值个数：

In [ ]:
cleaned_data["Country"] = cleaned_data["Country"].replace({"USA": "United States"})

In [ ]:
len(cleaned_data[cleaned_data["Country"] == "USA"])

把`Country`变量值`"UK"`、`"U.K."`替换为`"United Kingdom"`，并检查替换后`"UK"`和`"U.K."`变量值个数：

In [ ]:
cleaned_data["Country"] = cleaned_data["Country"].replace({"UK": "United Kingdom", "U.K.": "United Kingdom"})


In [ ]:
print(len(cleaned_data[cleaned_data["Country"] == "UK"]))
print(len(cleaned_data[cleaned_data["Country"] == "U.K."]))

把`Quantity`变量值为负数的观察值删除，并检查替换后`Quantity`变量值为负数的个数：

In [ ]:
cleaned_data = cleaned_data[cleaned_data["Quantity"] >= 0]

In [ ]:
len(cleaned_data[cleaned_data["Quantity"] < 0])

把`UnitPrice`变量值为负数的观察值删除，并检查替换后`UnitPrice`变量值为负数的个数：

In [ ]:
cleaned_data = cleaned_data[cleaned_data["UnitPrice"] >= 0]

In [ ]:
len(cleaned_data[cleaned_data["UnitPrice"] < 0])

## 保存清理后的数据

完成数据清理后，把干净整齐的数据保存到新的文件里，文件名为`e_commerce_cleaned.csv`。

In [ ]:
cleaned_data.head()

In [ ]:
cleaned_data.to_csv("e_commerce_cleaned.csv", index=False)

In [ ]:
pd.read_csv("e_commerce_cleaned.csv").head()